- #### Download data from here:<br>
https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet <br>
https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_05.parquet <br>
https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_06.parquet


### Install Python Libraries


#### Run this command in your python (virtual) environment


#### <i> `pip install duckdb magic_duckdb polars plotly_express nbformat --quiet --user` </i>


#### 1. duckdb:

DuckDB is an embedded analytical SQL database for Python.
It is designed for fast query execution and low memory usage.
Developers often use it for data analysis, data manipulation, and reporting.
You can find more information about DuckDB on their official website .

#### 2. magic_duckdb:

magic_duckdb is a Python package that provides Jupyter Notebook magic commands for interacting with DuckDB.
It allows you to run SQL queries against DuckDB directly within a Jupyter Notebook.
You can explore its usage and documentation in the GitHub repository: <br> https://github.com/iqmo-org/magic_duckdb, or here:<br>https://pypi.org/project/magic-duckdb/

#### 3. Polars:

Polars is a fast DataFrame library for Python and Rust.
It is designed for big data processing and provides a similar interface to Pandas.
Polars is particularly useful for working with large datasets efficiently.
To learn more about Polars visit the official website:<br> https://pola.rs/

#### 4. Plotly Express:

Plotly Express is a high-level Python visualization library built on top of Plotly.
It simplifies the creation of interactive plots, charts, and graphs.
With Plotly Express, you can quickly generate visualizations without writing extensive code.
Explore its capabilities in the official documentation: <br>https://plotly.com/python/plotly-express/

#### 5. nbformat:

nbformat is a Python library for working with Jupyter Notebook file formats.
It allows you to read, write, and manipulate Jupyter Notebook files programmatically.
Developers often use it for tasks like converting notebooks to different formats or extracting cell content.
You can find more details in the official documentation: <br>https://nbformat.readthedocs.io/en/latest/ and here: <br>https://pypi.org/project/nbformat/


#### Use the `--quiet` flag to suppress output during installation, and the `--user` flag to install the packages in your user-specific Python environment.


In [ ]:
# .torenv\Scripts\Activate.ps1

In [ ]:
import duckdb
import pandas as pd
#import plotly_express as px

%load_ext magic_duckdb

#### We already installed and loaded the duckdb magic in our notebook. Let us take advantage of it <br>so that we don't repeate `duckdb.sql` ever time. <br>Instead we can use: <br>

- `%dql` for single line queries and:<br>
- `%%dql` for multi-line queries instead


##### Because we are using the magic_duckdb extension, our queries will return a Pandas DataFrame, <br> bringing the entire query result into memory.

##### We can avoid this by setting the type of return by using `"-t"` followed by the type, choosing from "df", "arrow", "pl", "describe", "show" and "relation".


#### We can also query files over the internet using duckdb's `httpfs` extension

In [ ]:
%%dql -t df
LOAD  httpfs;
SELECT format('{:,}', COUNT(*)) as count 
FROM 'https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet'

In [ ]:
%%time
%%dql -t show
SELECT * FROM 'https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet' LIMIT 5;

#### Let us inspect the internet parquet file from our local duckdb using the httpfs extension


In [ ]:
%%time
%%dql
SELECT COUNT(*) AS RowCount, 
       AVG(passenger_count) AS avg_number_of_passengers,
       AVG(trip_distance) AS avg_trip_distance,
       MAX(trip_distance) AS max_trip_distance,
       AVG(fare_amount) AS avg_fare_amount,
       MAX(fare_amount) AS max_fare_amount,
       AVG(tip_amount) AS avg_tip_amount,
       MAX(tip_amount) AS max_tip_amount FROM 'https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet';

#### Find current working directory path

In [ ]:
%pwd 

#### Let us create a local copy of the internet taxi_2019_04 parquet file

In [ ]:
%%time
%%dql
COPY (SELECT * FROM READ_PARQUET('https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet'))
      TO 'taxi_2019_04.parquet' (FORMAT 'parquet');

#### Let us verify rowcount in the newly created parquet file

In [ ]:
%dql -t df SELECT format('{:,}', COUNT(*)) as count FROM 'taxi_2019_04.parquet';

#### We can also exclude columns using an matching expression

In [ ]:
%%dql -t df
SELECT COLUMNS(c -> c LIKE 'to%') FROM 'taxi_2019_04.parquet' limit 5;

#### Let us look at the datatypes in the parquet file - <i>Note the duckdb sql syntax </i>

In [ ]:
%%dql
DESCRIBE FROM 'taxi_2019_04.parquet';

### Let us lok at the Parquet file


#### Let us take a look at the parquet file's metadata

In [ ]:
%%dql 
SELECT *
FROM parquet_metadata('taxi_2019_04.parquet')

#### Let us limit the number of columns to what we are looking for

In [ ]:
%%dql 
SELECT file_name, total_compressed_size, total_uncompressed_size 
FROM parquet_metadata('taxi_2019_04.parquet')

#### Let us see how much space the parquet file takes up on disk

In [ ]:
%%dql -t df
SELECT 
       format('{:,}', CAST(ROUND((sum(total_compressed_size) / (1024))) AS INT)) as compressed_KB,
       format('{:,}', CAST(ROUND((sum(total_uncompressed_size) / (1024))) AS INT)) as uncompressed_KB,
       format('{:,}', CAST(ROUND((sum(total_compressed_size) / (1024 * 1024))) AS INT)) as compressed_MB,
       format('{:,}', CAST(ROUND((sum(total_uncompressed_size) / (1024 * 1024))) AS INT)) as uncompressed_MB,
       sum(total_compressed_size) / (1024 * 1024 * 1024) as compressed_GB,
       sum(total_uncompressed_size) / (1024 * 1024 * 1024) as uncompressed_GB    
FROM parquet_metadata('taxi_2019_04.parquet')

### Do not show any Pandas dataframe numbers with scientific notation


In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)  # Show 2 decimal places

#### Let us look at some aggregates from our data

In [ ]:
%%time
%%dql -t show
SELECT format('{:,}', COUNT(*)) AS RowCount, 
       AVG(passenger_count) AS avg_number_of_passengers,
       AVG(trip_distance) AS avg_trip_distance,
       MAX(trip_distance) AS max_trip_distance,
       AVG(fare_amount) AS avg_fare_amount,
       MAX(fare_amount) AS max_fare_amount,
       AVG(tip_amount) AS avg_tip_amount,
       MAX(tip_amount) AS max_tip_amount FROM 'taxi_2019_04.parquet';

#### Notice the extreme fare_amount value ($395,839.94)

#### We need to inspect that record. It could be a corrupt record.

In [ ]:
%dql SELECT * FROM 'taxi_2019_04.parquet' WHERE fare_amount = (SELECT MAX(fare_amount) FROM 'taxi_2019_04.parquet');

#### <i>As we suspected there is a problem with paying that fare_amount for a trip_distance of 0.00 miles. <br>Also notice the pickup_at, dropoff_at timestamps as well as pickup_location_id and dropoff_location_id.</i>

### If you wish to merge duckdb sql queries and python code, you should use the duckdb.sql('...') query method, 

In [ ]:
var1 = duckdb.sql(" SELECT COUNT(*) FROM 'taxi_2019_04.parquet' WHERE pickup_at BETWEEN '2019-04-10' AND '2019-04-12'");
print(var1)

### or use the `%dql -o <variablename>` method.

In [ ]:
%dql -o var2 SELECT COUNT(*) FROM 'taxi_2019_04.parquet' WHERE pickup_at BETWEEN '2019-04-10' AND '2019-04-12'
print(var2)

## More magic_duckdb

#### List available dql types 

In [ ]:
%dql --listtypes

#### List all tables used in the query

In [ ]:
%dql --tables  SELECT COUNT(*) As qCount FROM 'taxi_2019_04.parquet'

#### Get the connection created within DQL and use it directly

In [ ]:
con = %dql --getcon
display(con.sql("pragma version").df())

#### Create a table in an existing database `taxi_2019_04.db`

In [ ]:
dql_con = duckdb.connect("taxi_2019_04.db")
dql_test_con.sql("CREATE OR REPLACE TABLE xyz as select * from range(100)")

# -co sets the connection to an existing object
%dql -co dql_test_con
%dql select count(*) from xyz

#### We can also pass parameters to the query

In [ ]:
startdate = '2019-04-22'
enddate = '2019-04-25'
%dql -p startdate -p enddate -o outvar SELECT COUNT(*) As qCount FROM 'taxi_2019_04.parquet' WHERE pickup_at BETWEEN ? AND ?;
print(outvar)

### Create a new persistent duckdb database for the nyc_taxi_data


In [ ]:
con = duckdb.connect("taxi_2019_04_06.db") 
con.close()

#### Create a connection explicitly, and pass it explicitly to the dql connection. <br>dql by default will use the duckdb default connection

#### Create a table from a our taxi_2019_04 Parquet file

In [ ]:
con = duckdb.connect("taxi_2019_04_06.db")
con.execute("CREATE OR REPLACE TABLE taxi_trips AS (SELECT * FROM 'taxi_2019_*.parquet');")

#### The `-co` sets the connection to an existing database object

In [ ]:
%dql -co con
%dql SELECT format('{:,}', COUNT(*)) AS count FROM taxi_trips;

#### We can also convert to pandas dataFrame directly from your duckdb query<br> by adding the ".df()" function
#### Query the first 10 rows of the newly created taxi_trips data. <br> ####
#### <i>Notice that I use duckdb's to pandas dataframe `df()` at the end of the query.</i>


In [ ]:
con.sql("SELECT * FROM taxi_trips LIMIT 3").df()

## Analyzing Data with DuckDB


### SQL queries using DuckDB:

To run SQL queries in DuckDB we can directly use ".sql", no need to create a connection to ":memory:"

Every DataFrame inside this notebook will be instantly available for DuckDB to make SQL queries against.


In [ ]:
%dql SUMMARIZE SELECT * FROM taxi_trips WHERE vendor_id = 1;

#### Let us look at the Datatypes in the table

In [ ]:
%dql -co con
%dql DESCRIBE taxi_trips;

#### Let us test out query speed in the new duckdb table

In [ ]:
%%time
%%dql
SELECT format('{:,}', COUNT(*)) AS RowCount, 
       AVG(passenger_count) AS avg_number_of_passengers,
       AVG(trip_distance) AS avg_trip_distance,
       MAX(trip_distance) AS max_trip_distance,
       AVG(fare_amount) AS avg_fare_amount,
       MAX(fare_amount) AS max_fare_amount,
       AVG(tip_amount) AS avg_tip_amount,
       MAX(tip_amount) AS max_tip_amount FROM taxi_trips;

#### END OF FILE
